In [8]:
!pip install kagglehub pandas scikit-learn fairlearn aif360 --quiet

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate,
    count
)
from datetime import datetime

# Load dataset
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\compas-scores-raw.csv")
print("Original shape:", df.shape)

# Filtering
df = df[
    (df['IsCompleted'] == 1) &
    (df['ScoreText'].isin(['Low', 'Medium', 'High']))
]

# Target: 1 = reoffended (Medium or High), 0 = Low
df['two_year_recid'] = df['ScoreText'].map({'Low': 0, 'Medium': 1, 'High': 1})

# Keep only African-American and Caucasian
df = df[df['Ethnic_Code_Text'].isin(['African-American', 'Caucasian'])]
df['race_binary'] = df['Ethnic_Code_Text'].map({'Caucasian': 0, 'African-American': 1})
df['sex_binary'] = df['Sex_Code_Text'].map({'Male': 1, 'Female': 0})

# Derive age from DateOfBirth and Screening_Date
df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'], errors='coerce')
df['Screening_Date'] = pd.to_datetime(df['Screening_Date'], errors='coerce')
df['age'] = (df['Screening_Date'] - df['DateOfBirth']).dt.days // 365

# Drop rows with missing values in new fields
df = df.dropna(subset=['age', 'LegalStatus', 'CustodyStatus', 'RecSupervisionLevel'])

# Encode categorical features
df = pd.get_dummies(df, columns=['LegalStatus', 'CustodyStatus', 'RecSupervisionLevel'], drop_first=True)

# Select features
features = ['sex_binary', 'age'] + [col for col in df.columns if col.startswith('LegalStatus_') or col.startswith('CustodyStatus_') or col.startswith('RecSupervisionLevel_')]
X = df[features]
y = df['two_year_recid']
protected = df['race_binary']

# Normalize features
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Train-test split
X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
    X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
)

# Check outputs
print("Final shape:", X_train.shape)
print("y_train class balance:\n", y_train.value_counts(normalize=True))
print("Protected attribute (train):\n", prot_train.value_counts())


Original shape: (60843, 28)


C:\Users\Arhamsoft\AppData\Local\Temp\ipykernel_29564\4235935015.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'], errors='coerce')
C:\Users\Arhamsoft\AppData\Local\Temp\ipykernel_29564\4235935015.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Screening_Date'] = pd.to_datetime(df['Screening_Date'], errors='coerce')


Final shape: (34136, 16)
y_train class balance:
 two_year_recid
0    0.652625
1    0.347375
Name: proportion, dtype: float64
Protected attribute (train):
 race_binary
1    18949
0    15187
Name: count, dtype: int64


In [2]:
# Train Logistic Regression
lr_model = LogisticRegression(solver='liblinear', random_state=42)
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

# Performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness metrics
metric_frame = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate, "count": count},
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=prot_test
)

print("\nPer-group Accuracy and Selection Rate:")
print(metric_frame.by_group)

# 📏 Fairness differences
spd = demographic_parity_difference(y_test, y_pred, sensitive_features=prot_test)
eod = equalized_odds_difference(y_test, y_pred, sensitive_features=prot_test)
sr = metric_frame.by_group["selection_rate"]
dir_ratio = sr[1] / sr[0] if sr[0] != 0 else float("inf")

print("\n⚖️ Fairness Metrics:")
print(f"Statistical Parity Difference  (SPD): {spd:.4f}")
print(f"Equal Opportunity Difference  (EOD): {eod:.4f}")
print(f"Disparate Impact Ratio (DIR)       : {dir_ratio:.4f}")

📈 Performance Metrics:
Accuracy : 0.7994
Precision: 0.6792
Recall   : 0.8007
F1 Score : 0.7349

📊 Per-group Accuracy and Selection Rate:
             accuracy  selection_rate   count
race_binary                                  
0            0.829735        0.260718  6578.0
1            0.774618        0.530982  8053.0

⚖️ Fairness Metrics:
Statistical Parity Difference  (SPD): 0.2703
Equal Opportunity Difference  (EOD): 0.1759
Disparate Impact Ratio (DIR)       : 2.0366


In [3]:
# Imports
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate,
    count
)

# Step 1: Convert to AIF360 BinaryLabelDataset
# Define privileged/unprivileged groups for race
privileged_groups = [{'race_binary': 0}]
unprivileged_groups = [{'race_binary': 1}]

# Combine back into a single training DataFrame with labels and protected attributes
train_df = X_train.copy()
train_df['label'] = y_train.values
train_df['race_binary'] = prot_train.values

# Convert to AIF360 dataset
train_bld = BinaryLabelDataset(
    df=train_df,
    label_names=['label'],
    protected_attribute_names=['race_binary']
)

# Step 2: Apply Reweighing
rw = Reweighing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups
)
rw.fit(train_bld)
train_bld_rw = rw.transform(train_bld)

# Extract features and labels
X_train_rw = pd.DataFrame(train_bld_rw.features, columns=train_bld_rw.feature_names)
# Drop race_binary so features match X_test
X_train_rw = X_train_rw.drop(columns=["race_binary"])

y_train_rw = train_bld_rw.labels.ravel()
sample_weights = train_bld_rw.instance_weights

# Step 3: Train weighted Logistic Regression
lr_rw_model = LogisticRegression(solver='liblinear', random_state=42)
lr_rw_model.fit(X_train_rw, y_train_rw, sample_weight=sample_weights)

# Step 4: Predict on test set
y_pred_rw = lr_rw_model.predict(X_test)

# Performance metrics
accuracy = accuracy_score(y_test, y_pred_rw)
precision = precision_score(y_test, y_pred_rw)
recall = recall_score(y_test, y_pred_rw)
f1 = f1_score(y_test, y_pred_rw)

print("Reweighed Model Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness evaluation
metric_frame_rw = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate, "count": count},
    y_true=y_test,
    y_pred=y_pred_rw,
    sensitive_features=prot_test
)

print("\nReweighed Group Metrics:")
print(metric_frame_rw.by_group)

# Fairness Metrics
spd = demographic_parity_difference(y_test, y_pred_rw, sensitive_features=prot_test)
eod = equalized_odds_difference(y_test, y_pred_rw, sensitive_features=prot_test)
sr = metric_frame_rw.by_group["selection_rate"]
dir_ratio = sr[1] / sr[0] if sr[0] != 0 else float("inf")

print("\nReweighed Fairness Metrics:")
print(f"Statistical Parity Difference  (SPD): {spd:.4f}")
print(f"Equal Opportunity Difference  (EOD): {eod:.4f}")
print(f"Disparate Impact Ratio (DIR)       : {dir_ratio:.4f}")


pip install 'aif360[inFairness]'


📈 Reweighed Model Performance:
Accuracy : 0.7994
Precision: 0.6792
Recall   : 0.8007
F1 Score : 0.7349

📊 Reweighed Group Metrics:
             accuracy  selection_rate   count
race_binary                                  
0            0.829735        0.260718  6578.0
1            0.774618        0.530982  8053.0

⚖️ Reweighed Fairness Metrics:
Statistical Parity Difference  (SPD): 0.2703
Equal Opportunity Difference  (EOD): 0.1759
Disparate Impact Ratio (DIR)       : 2.0366


In [4]:
# Install Fairlearn if not done already
!pip install fairlearn --quiet

# Imports
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Create a ThresholdOptimizer (Equalized Odds)
eo = ThresholdOptimizer(
    estimator=lr_model,  # your already trained Logistic Regression
    constraints="equalized_odds",
    predict_method="predict_proba",  # required for probabilistic predictions
    prefit=True  # model is already trained
)

# Step 2: Fit the optimizer
eo.fit(X_train, y_train, sensitive_features=prot_train)

# Step 3: Predict using post-processed predictions
y_pred_eo = eo.predict(X_test, sensitive_features=prot_test)

# Performance metrics
accuracy = accuracy_score(y_test, y_pred_eo)
precision = precision_score(y_test, y_pred_eo)
recall = recall_score(y_test, y_pred_eo)
f1 = f1_score(y_test, y_pred_eo)

print("Equalized Odds Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Fairness metrics
from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    equalized_odds_difference,
    selection_rate,
    count
)

metric_frame_eo = MetricFrame(
    metrics={"accuracy": accuracy_score, "selection_rate": selection_rate, "count": count},
    y_true=y_test,
    y_pred=y_pred_eo,
    sensitive_features=prot_test
)

print("\nEqualized Odds Group Metrics:")
print(metric_frame_eo.by_group)

# Group fairness metrics
spd = demographic_parity_difference(y_test, y_pred_eo, sensitive_features=prot_test)
eod = equalized_odds_difference(y_test, y_pred_eo, sensitive_features=prot_test)
sr = metric_frame_eo.by_group["selection_rate"]
dir_ratio = sr[1] / sr[0] if sr[0] != 0 else float("inf")

print("\nEqualized Odds Fairness Metrics:")
print(f"Statistical Parity Difference  (SPD): {spd:.4f}")
print(f"Equal Opportunity Difference  (EOD): {eod:.4f}")
print(f"Disparate Impact Ratio (DIR)       : {dir_ratio:.4f}")



📈 Equalized Odds Performance:
Accuracy : 0.7667
Precision: 0.6437
Recall   : 0.7349
F1 Score : 0.6863

📊 Equalized Odds Group Metrics:
             accuracy  selection_rate   count
race_binary                                  
0            0.776072        0.340833  6578.0
1            0.758972        0.442071  8053.0

⚖️ Equalized Odds Fairness Metrics:
Statistical Parity Difference  (SPD): 0.1012
Equal Opportunity Difference  (EOD): 0.0130
Disparate Impact Ratio (DIR)       : 1.2970


In [10]:
# Adversarial Debiasing - Improved Fairness-Aware Training

# --- Reset TensorFlow graph ---
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.reset_default_graph()
sess = tf.Session()

from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- OPTIONAL: Expand feature set if needed (already scaled) ---
# If you previously filtered features to just 3, now add more if available
# Example:
# features = ['RawScore', 'DecileScore', 'sex_binary', 'CustodyStatus', ...]

# --- Rebuild combined dataframes ---
train_df = pd.concat([X_train, y_train.rename("two_year_recid"), prot_train.rename("race_binary")], axis=1).dropna()
test_df  = pd.concat([X_test, y_test.rename("two_year_recid"), prot_test.rename("race_binary")], axis=1).dropna()

# --- Convert to AIF360 BinaryLabelDataset format ---
train_bld = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=train_df,
    label_names=["two_year_recid"],
    protected_attribute_names=["race_binary"]
)

test_bld = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=test_df,
    label_names=["two_year_recid"],
    protected_attribute_names=["race_binary"]
)

# --- Train Adversarial Debiasing model ---
adv_debias = AdversarialDebiasing(
    privileged_groups=[{'race_binary': 0}],
    unprivileged_groups=[{'race_binary': 1}],
    scope_name='adv_debiasing_classifier',
    sess=sess,
    debias=True,
    num_epochs=100,
    batch_size=128,
    adversary_loss_weight=0.001   
)
adv_debias.fit(train_bld)

# --- Predict on test set ---
pred_bld = adv_debias.predict(test_bld)

# --- Extract values for evaluation ---
y_pred_adv = pred_bld.labels.ravel()
y_true_adv = test_bld.labels.ravel()
prot_attr_adv = test_bld.protected_attributes.ravel()

# --- Check prediction distribution (for debugging) ---
import numpy as np
print("\nPrediction distribution:")
print(np.unique(y_pred_adv, return_counts=True))

# --- Evaluate Performance Metrics ---
accuracy = accuracy_score(y_true_adv, y_pred_adv)
precision = precision_score(y_true_adv, y_pred_adv, zero_division=0)
recall = recall_score(y_true_adv, y_pred_adv, zero_division=0)
f1 = f1_score(y_true_adv, y_pred_adv, zero_division=0)

print("\nAdversarial Debiasing Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# --- Evaluate Fairness Metrics ---
metric_adv = ClassificationMetric(
    test_bld,
    pred_bld,
    unprivileged_groups=[{'race_binary': 1}],
    privileged_groups=[{'race_binary': 0}]
)

spd = metric_adv.statistical_parity_difference()
eod = metric_adv.equal_opportunity_difference()
dir_ratio = metric_adv.disparate_impact()

print("\n Adversarial Debiasing Fairness Metrics:")
print(f"Statistical Parity Difference  (SPD): {spd:.4f}")
print(f"Equal Opportunity Difference  (EOD): {eod:.4f}")
print(f"Disparate Impact Ratio (DIR)       : {dir_ratio:.4f}")

# --- Close TF session ---
sess.close()

epoch 0; iter: 0; batch classifier loss: 0.671116; batch adversarial loss: 0.678775
epoch 1; iter: 0; batch classifier loss: 0.614873; batch adversarial loss: 0.690508
epoch 2; iter: 0; batch classifier loss: 0.622071; batch adversarial loss: 0.683565
epoch 3; iter: 0; batch classifier loss: 0.686680; batch adversarial loss: 0.680954
epoch 4; iter: 0; batch classifier loss: 0.656803; batch adversarial loss: 0.688287
epoch 5; iter: 0; batch classifier loss: 0.592808; batch adversarial loss: 0.658564
epoch 6; iter: 0; batch classifier loss: 0.581240; batch adversarial loss: 0.698688
epoch 7; iter: 0; batch classifier loss: 0.602844; batch adversarial loss: 0.672319
epoch 8; iter: 0; batch classifier loss: 0.607793; batch adversarial loss: 0.659699
epoch 9; iter: 0; batch classifier loss: 0.568660; batch adversarial loss: 0.608240
epoch 10; iter: 0; batch classifier loss: 0.669990; batch adversarial loss: 0.588737
epoch 11; iter: 0; batch classifier loss: 0.600053; batch adversarial loss: